# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [2]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

In [3]:
!pip install torch

In [4]:
!pip install torchvision

In [5]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3
import os

In [6]:
bucket= 'dog-images-mle'
print("Default Bucket: {}".format(bucket))

region = 'us-east-1'## TODO: fill in
print("AWS Region: {}".format(region))

role = sagemaker.get_execution_role()## TODO: fill in
print("RoleArn: {}".format(role))

Default Bucket: dog-images-mle
AWS Region: us-east-1
RoleArn: arn:aws:iam::439568397364:role/service-role/AmazonSageMaker-ExecutionRole-20221228T214147


In [7]:
s3 = boto3.resource('s3')
Bucket = s3.Bucket(bucket)
category = 'valid'
files = [obj.key for obj in Bucket.objects.all() if obj.key.__contains__(category)]

## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [9]:
import sagemaker
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
from torchvision import transforms
from sagemaker.pytorch import PyTorch

In [9]:
#TODO: Declare your HP ranges, metrics etc.

In [10]:
#TODO: Create estimators for your HPs

estimator = PyTorch(
    entry_point="hpo.py",
    role=role,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.g4dn.xlarge"
)

hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.1),
    "batch_size": CategoricalParameter([32, 64, 128]),
    "epochs": IntegerParameter(2, 4)
}

objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Test set: Average loss: ([0-9\\.]+)"}]

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=4,
    max_parallel_jobs=2,
    objective_type=objective_type
)

In [11]:
# TODO: Fit your HP Tuner
tuner.fit(wait=True) # TODO: Remember to include your data channels

..................................................................................................................................................................................................................................!


In [12]:
tuner.best_training_job()

'pytorch-training-230116-1632-003-cfff4a1a'

In [13]:
# TODO: Get the best estimators and the best HPs

best_estimator = tuner.best_estimator()

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()


2023-01-16 16:49:15 Starting - Found matching resource for reuse
2023-01-16 16:49:15 Downloading - Downloading input data
2023-01-16 16:49:15 Training - Training image download completed. Training in progress.
2023-01-16 16:49:15 Uploading - Uploading generated training model
2023-01-16 16:49:15 Completed - Resource retained for reuse


{'_tuning_objective_metric': '"average test loss"',
 'batch_size': '"64"',
 'epochs': '2',
 'lr': '0.0027817712552108526',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"pytorch-training-2023-01-16-16-32-12-186"',
 'sagemaker_program': '"hpo.py"',
 'sagemaker_region': '"us-east-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-439568397364/pytorch-training-2023-01-16-16-32-12-186/source/sourcedir.tar.gz"'}

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [10]:
from sagemaker.debugger import Rule, ProfilerRule, rule_configs
from sagemaker.debugger import DebuggerHookConfig, ProfilerConfig, FrameworkProfile
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts
import IPython

[2023-01-18 01:32:53.972 1-8-1-cpu-py36-ml-t3-medium-05a4a7868130c7575335c53b16c7:32 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None


In [11]:
hyperparameters = {'batch_size': '128', 'epochs': '2', 'lr': '0.001425456445464047'}

In [12]:
# TODO: Set up debugging and profiling rules and hooks
rules = [
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
]

from sagemaker.debugger import DebuggerHookConfig, ProfilerConfig, FrameworkProfile

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10)
)
debugger_config = DebuggerHookConfig(
    hook_parameters={"train.save_interval": "100", "eval.save_interval": "10"}
)


In [13]:
# TODO: Create and fit an estimator

estimator = PyTorch(
    entry_point='train_model.py',
    role=role,
    instance_count=1,
    instance_type="ml.g4dn.xlarge",
    hyperparameters=hyperparameters,
    framework_version="1.8",
    py_version="py36",
    ## Debugger parameters
    profiler_config=profiler_config,
    debugger_hook_config=debugger_config,
    rules=rules,
    source_dir="code"
)

In [14]:
estimator.fit(wait=True)

2023-01-18 01:33:15 Starting - Starting the training job...
2023-01-18 01:33:42 Starting - Preparing the instances for trainingLossNotDecreasing: InProgress
VanishingGradient: InProgress
Overfit: InProgress
Overtraining: InProgress
PoorWeightInitialization: InProgress
LowGPUUtilization: InProgress
ProfilerReport: InProgress
......
2023-01-18 01:34:42 Downloading - Downloading input data...
2023-01-18 01:35:07 Training - Downloading the training image.....................
2023-01-18 01:38:44 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-01-18 01:38:38,750 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-01-18 01:38:38,778 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-01-18 01:38:38,781 sagemaker_pytorch_container.training INFO     Invoking user training scri

In [15]:
#session = boto3.session.Session()
#region = session.region_name

training_job_name = estimator.latest_training_job.name
print(f"Training jobname: {training_job_name}")
print(f"Region: {region}")

trial = create_trial(estimator.latest_job_debugger_artifacts_path())

print(trial.tensor_names())
print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.TRAIN)))
print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.EVAL)))

tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)

rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

! aws s3 ls {rule_output_path} --recursive
! aws s3 cp {rule_output_path} ./ --recursive

# get the autogenerated folder name of profiler report
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

IPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")

Training jobname: pytorch-training-2023-01-18-01-33-15-174
Region: us-east-1
[2023-01-18 01:46:59.098 1-8-1-cpu-py36-ml-t3-medium-05a4a7868130c7575335c53b16c7:32 INFO s3_trial.py:42] Loading trial debug-output at path s3://sagemaker-us-east-1-439568397364/pytorch-training-2023-01-18-01-33-15-174/debug-output
[2023-01-18 01:46:59.332 1-8-1-cpu-py36-ml-t3-medium-05a4a7868130c7575335c53b16c7:32 INFO trial.py:198] Training has ended, will refresh one final time in 1 sec.
[2023-01-18 01:47:00.353 1-8-1-cpu-py36-ml-t3-medium-05a4a7868130c7575335c53b16c7:32 INFO trial.py:210] Loaded all steps
['CrossEntropyLoss_output_0', 'gradient/ResNet_fc.0.bias', 'gradient/ResNet_fc.0.weight', 'layer1.0.relu_input_0', 'layer1.0.relu_input_1', 'layer1.1.relu_input_0', 'layer1.1.relu_input_1', 'layer2.0.relu_input_0', 'layer2.0.relu_input_1', 'layer2.1.relu_input_0', 'layer2.1.relu_input_1', 'layer3.0.relu_input_0', 'layer3.0.relu_input_1', 'layer3.1.relu_input_0', 'layer3.1.relu_input_1', 'layer4.0.relu_in

You will find the profiler report in s3://sagemaker-us-east-1-439568397364/pytorch-training-2023-01-18-01-33-15-174/rule-output
2023-01-18 01:46:31     412986 pytorch-training-2023-01-18-01-33-15-174/rule-output/ProfilerReport/profiler-output/profiler-report.html
2023-01-18 01:46:30     268546 pytorch-training-2023-01-18-01-33-15-174/rule-output/ProfilerReport/profiler-output/profiler-report.ipynb
2023-01-18 01:46:25        543 pytorch-training-2023-01-18-01-33-15-174/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json
2023-01-18 01:46:25       4870 pytorch-training-2023-01-18-01-33-15-174/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
2023-01-18 01:46:25       2014 pytorch-training-2023-01-18-01-33-15-174/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json
2023-01-18 01:46:25        130 pytorch-training-2023-01-18-01-33-15-174/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.jso

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

## Model Deploying

In [16]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

In [17]:
from sagemaker.pytorch.model import PyTorchModel

In [77]:
estimator_trained = PyTorchModel(
    model_data = 's3://sagemaker-us-east-1-439568397364/pytorch-training-2023-01-18-01-33-15-174/output/model.tar.gz',
    entry_point='inference.py',
    framework_version="1.8",
    py_version="py36",
    source_dir="code",
    role=role
)

In [81]:
# TODO: Deploy your model to an endpoint

predictor=estimator_trained.deploy(initial_instance_count=1, 
                           instance_type="ml.t2.medium")
                              #serializer=JSONSerializer(),
                        #deserializer=JSONDeserializer())
                          #data_capture_config=data_capture_config) # TODO: Add your deployment configuration like instance type and number of instances

INFO:sagemaker:Creating model with name: pytorch-inference-2023-01-18-03-38-21-636
INFO:sagemaker:Creating endpoint with name pytorch-inference-2023-01-18-03-38-22-306


----------!

In [20]:
from PIL import Image
import torchvision.transforms as transforms

In [131]:
import numpy as np
import pandas as pd

In [132]:
# TODO: Run an prediction on the endpoint

fname = 'dogImages/valid/077.Gordon_setter/Gordon_setter_05284.jpg'
image = Image.open(fname) # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(np.array(image))

In [113]:
labels = np.array([s.split('valid/')[1].split('/')[0].split('.') for s in files if len(s.split('valid/'))>1])

In [126]:
label_df = pd.DataFrame({'number':labels[:,0], 'label':labels[:,1]}).drop_duplicates().reset_index(drop=True)

In [133]:
label_df.iloc[np.argmax(response)]

number                    006
label     American_eskimo_dog
Name: 5, dtype: object

In [80]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: pytorch-inference-2023-01-18-03-29-19-233
INFO:sagemaker:Deleting endpoint with name: pytorch-inference-2023-01-18-03-29-19-233
